# V12: Strong Ensemble with Early Stopping

A robust ensemble approach combining three gradient boosting models with heavy regularization and strategic early stopping to prevent overfitting on the 700K training samples.

**Key Features:**
- 3-model ensemble: XGBoost, LightGBM, CatBoost
- External feature encoding from 100K diabetes dataset
- Medical domain features (BMI, BP, non-HDL)
- 10-Fold Stratified Cross-Validation
- Heavy regularization (L1=1.5, L2=2.0-2.2)
- Weighted ensemble blending (50/35/15)
- Probability clipping for calibration

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

print("V12")

## 2. Load the Data

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s5e12/train.csv')
test  = pd.read_csv('/kaggle/input/playground-series-s5e12/test.csv')
sub   = pd.read_csv('/kaggle/input/playground-series-s5e12/sample_submission.csv')
orig  = pd.read_csv('/kaggle/input/diabetes-health-indicators-dataset/diabetes_dataset.csv')

TARGET = 'diagnosed_diabetes'

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

## 3. External Encoding

Mean and count (log-scaled) encodings from the 100K external diabetes dataset.

In [ ]:
base_cols = [c for c in train.columns if c not in ['id', TARGET]]
encoded = []

for col in base_cols:
    # Mean encoding
    mapping = orig.groupby(col)[TARGET].mean()
    train[f"enc_mean_{col}"] = train[col].map(mapping)
    test[f"enc_mean_{col}"]  = test[col].map(mapping)
    encoded.append(f"enc_mean_{col}")
    
    # Smoothed count encoding
    cnt = orig.groupby(col).size()
    train[f"enc_cnt_{col}"] = train[col].map(cnt).fillna(1)
    test[f"enc_cnt_{col}"]  = test[col].map(cnt).fillna(1)
    train[f"enc_cnt_{col}"] = np.log1p(train[f"enc_cnt_{col}"])
    test[f"enc_cnt_{col}"]  = np.log1p(test[f"enc_cnt_{col}"])
    encoded.append(f"enc_cnt_{col}")

print(f"Generated {len(encoded)} external features")

## 4. Safe & Strong Features

In [ ]:
train['bmi_cat'] = pd.cut(train['bmi'], bins=[0,18.5,25,30,999], labels=[0,1,2,3]).astype('int')
test['bmi_cat']  = pd.cut(test['bmi'],  bins=[0,18.5,25,30,999], labels=[0,1,2,3]).astype('int')

train['bp_cat'] = 0
train.loc[(train['systolic_bp']>=140)|(train['diastolic_bp']>=90), 'bp_cat'] = 2
train.loc[((train['systolic_bp']>=120)&(train['systolic_bp']<140))|
          ((train['diastolic_bp']>=80)&(train['diastolic_bp']<90)), 'bp_cat'] = 1

test['bp_cat'] = 0
test.loc[(test['systolic_bp']>=140)|(test['diastolic_bp']>=90), 'bp_cat'] = 2
test.loc[((test['systolic_bp']>=120)&(test['systolic_bp']<140))|
         ((test['diastolic_bp']>=80)&(test['diastolic_bp']<90)), 'bp_cat'] = 1

train['non_hdl'] = train['cholesterol_total'] - train['hdl_cholesterol']
test['non_hdl']  = test['cholesterol_total'] - test['hdl_cholesterol']

print("Medical features created")

## 5. Final Features + Label Encode

In [ ]:
features = base_cols + ['bmi_cat', 'bp_cat', 'non_hdl'] + encoded

# Fill NaNs from encoding
for f in encoded:
    train[f] = train[f].fillna(train[f].median())
    test[f]  = test[f].fillna(train[f].median())

X      = train[features].copy()
y      = train[TARGET]
X_test = test[features].copy()

# Label encode categoricals
cat_cols = ['bmi_cat', 'bp_cat'] + train.select_dtypes('object').columns.tolist()
for col in cat_cols:
    if col in X.columns:
        le = LabelEncoder()
        X[col]      = le.fit_transform(X[col].astype(str))
        X_test[col] = le.transform(X_test[col].astype(str))

print(f"Total features: {X.shape[1]}")

## 6. 10-Fold Strong Ensemble + Calibration

Heavy regularization strategy:
- **XGBoost**: max_depth=6, L1=1.5, L2=2.0, learning_rate=0.01
- **LightGBM**: max_depth=7, L1=1.5, L2=2.2, learning_rate=0.01
- **CatBoost**: depth=7, L2=6.0, learning_rate=0.01
- **Ensemble weights**: 50% XGB + 35% LGBM + 15% CB

In [ ]:
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

oof = np.zeros(len(X))
preds = np.zeros(len(X_test))

print(f"\nStarting {n_splits}-fold training...\n")

for fold, (trn_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"Fold {fold}/{n_splits}", end=" → ")
    
    X_trn, X_val = X.iloc[trn_idx], X.iloc[val_idx]
    y_trn, y_val = y.iloc[trn_idx], y.iloc[val_idx]

    # XGBoost – heavy regularization
    model1 = xgb.XGBClassifier(
        n_estimators=5000,
        max_depth=6,
        learning_rate=0.01,
        subsample=0.8,
        colsample_bytree=0.6,
        reg_alpha=1.5,
        reg_lambda=2.0,
        random_state=42,
        tree_method='hist',
        n_jobs=-1,
        verbosity=0
    )
    model1.fit(X_trn, y_trn,
               eval_set=[(X_val, y_val)],
               early_stopping_rounds=250,
               verbose=False)

    # LightGBM – heavy regularization
    model2 = lgb.LGBMClassifier(
        n_estimators=5000,
        max_depth=7,
        learning_rate=0.01,
        num_leaves=48,
        subsample=0.8,
        colsample_bytree=0.6,
        reg_alpha=1.5,
        reg_lambda=2.2,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    model2.fit(X_trn, y_trn,
               eval_set=[(X_val, y_val)],
               callbacks=[lgb.early_stopping(250, verbose=False)])

    # CatBoost
    model3 = cb.CatBoostClassifier(
        iterations=5000,
        depth=7,
        learning_rate=0.01,
        l2_leaf_reg=6.0,
        random_seed=42,
        verbose=False,
        early_stopping_rounds=250
    )
    model3.fit(X_trn, y_trn, eval_set=(X_val, y_val), verbose=False)

    # Blend (best weights found by hill climbing)
    val_pred = (model1.predict_proba(X_val)[:,1] * 0.50 +
                model2.predict_proba(X_val)[:,1] * 0.35 +
                model3.predict_proba(X_val)[:,1] * 0.15)

    oof[val_idx] = val_pred
    auc = roc_auc_score(y_val, val_pred)
    print(f"AUC = {auc:.6f}")

    # Test preds
    preds += (model1.predict_proba(X_test)[:,1] * 0.50 +
              model2.predict_proba(X_test)[:,1] * 0.35 +
              model3.predict_proba(X_test)[:,1] * 0.15) / n_splits

print(f"\nFinal CV AUC: {roc_auc_score(y, oof):.6f}")

## 7. Final Submission

Apply probability clipping to reduce overconfidence in extreme predictions.

In [ ]:
final_pred = preds
final_pred = np.clip(final_pred, 0.01, 0.99)  # reduce overconfidence

sub[TARGET] = final_pred
sub.to_csv('submission.csv', index=False)

print("\nsubmission.csv saved!")
print(f"Mean prediction: {final_pred.mean():.5f}")
print(f"Min prediction: {final_pred.min():.5f}")
print(f"Max prediction: {final_pred.max():.5f}")

print("\nFirst few predictions:")
sub.head()

## Summary

**V12 Architecture:**
- 10-Fold Stratified Cross-Validation
- 3-model ensemble with heavy regularization
- 75 total features (24 base + 3 medical + 48 external)
- Early stopping with 250 rounds patience
- Weighted blending (50/35/15)
- Probability clipping for calibration
- Expected CV AUC: ~0.731

V12 represents a strong intermediate approach balancing model diversity with rigorous regularization.